In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import gensim
import re
import pickle
import string
import nltk
from functools import partial

from nltk.corpus import stopwords
sw = stopwords.words('english')
puncts = re.compile('[%s]' % re.escape(string.punctuation))
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
data = pd.read_pickle('../input/simplenormal-wikipedia-sections/wikipedia_sections.pkl.xz')['text'].tolist()
if not data[-1]:
    del(data[-1])
data = data[int(0.75*len(data)):]

In [ ]:
def clean_text(text):
    text = text.replace('\n', ' ')
    text = text.replace('\r', ' ')
    text = text.replace('\t', ' ')
    text = text.strip(' ')
    text = re.sub('\s+', ' ', text)
    text = text.encode('ascii','ignore').decode('utf-8')
    text = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)', "", text)
    text = re.sub(r'[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)', "", text)
    text = re.sub('[A-Za-z0-9_.]+@[A-Za-z0-9_.]+', '', text)
    text = re.sub(r'#[a-zA-Z0-9_.]+', '', text)
    text = re.sub('[0-9]+', '', text)
    text = puncts.sub(' ', text)
    text = re.sub('  ',' ',text)
    text = text.lower()
    text = text.split()
    text = [lemmatizer.lemmatize(i) for i in text if i not in sw]
    return text

def clean_batch(texts, clean_function):
    return list(map(clean_function, texts))

In [ ]:
data = clean_batch(data, clean_text)

In [ ]:
def load_file(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    return data

In [ ]:
bigrams = load_file('../input/ngrams-gensim/bigrams.pkl')
bigrams.add_vocab(data)
bigrams = bigrams.freeze()
with open('./bigrams.pkl', 'wb') as f:
    pickle.dump(bigrams, f)

In [ ]:
trigrams = load_file('../input/ngrams-gensim/trigrams.pkl')
trigrams.add_vocab(bigrams[data])
trigrams = trigrams.freeze()
with open('./trigrams.pkl', 'wb') as f:
    pickle.dump(trigrams, f)